In [19]:
##Healthy Individual

import pycxsimulator
from pylab import *

Recoveries = 0
Deaths = 0
Result = False
Results = []

width = 100
height = 100

initialPathogenPopulation = 10
pathogenReproductionRate = 0.02
pathogenPopulationLimit = 10**10
pathogenNoiseLevel = 3

initialMarkedPopulation = 0
markedReproductionRate = .02
markedPopulationLimit = 10**10
markedNoiseLevel = 3

helperPopulationLimit = [150,175]
initialHelperPopulation = helperPopulationLimit[0]
helperReproductionLimit = 0.2 ##Not presently used
helperNoiseLevel = 2.5
helperHungerLimit = 10

initialKillerPopulation = 50
killerReproductionLimit = 25
killerPopulationLimit = 100
killerNoiseLevel = 2.5
killerHungerLimit = 10

initialAntibodyPopulation = 50
antibodyReproductionLimit = 25
antibodyPopulationLimit = 100
antibodyNoiseLevel = 2.5
antibodyHungerLimit = 10

collisionDistance = 2.32
CDsquared = collisionDistance ** 2
toBeRemoved = -1

def initialize():
    global time, pathogens, killers, pathogenData, killerData, helpers, helperData, marked, markedData, antibodies ,antibodyData, Result

    time = 0
    Result = False
    
    pathogens = []
    for i in range(initialPathogenPopulation):
        pathogens.append([uniform(0, width), uniform(0, height),0])

    killers = []
    for i in range(initialKillerPopulation):
        killers.append([uniform(0, width), uniform(0, height), 0])
        
    helpers = []
    for i in range(initialHelperPopulation):
        helpers.append([uniform(0, width), uniform(0, height),round(10*random())])   
        
    marked = []
    for i in range(initialMarkedPopulation): #Can Probably be Deleted, Only Useful if starting simulation mid-infection
        Marked.append([uniform(0, width), uniform(0, height),0]) #Can Probably be Deleted, Only Useful if starting simulation mid-infection
        
    antibodies = []
    for i in range(initialAntibodyPopulation):
        antibodies.append([uniform(0, width), uniform(0, height), 0]) 

    pathogenData = [initialPathogenPopulation]
    killerData = [initialKillerPopulation]
    helperData = [initialHelperPopulation]
    markedData = [initialMarkedPopulation]
    antibodyData = [initialAntibodyPopulation]

def observe():
    subplot(1, 2, 1)
    cla()
    if pathogens != []:
        x = [ag[0] for ag in pathogens]
        y = [ag[1] for ag in pathogens]
        scatter(x, y, color = 'red')
    if marked != []:
        x = [ag[0] for ag in marked]
        y = [ag[1] for ag in marked]
        scatter(x, y, color = 'pink')
    if killers != []:
        x = [ag[0] for ag in killers]
        y = [ag[1] for ag in killers]
        scatter(x, y, color = 'green')
    if helpers != []:
        x = [ag[0] for ag in helpers]
        y = [ag[1] for ag in helpers]
        scatter(x, y, color = 'blue')
    if antibodies != []:
        x = [ag[0] for ag in antibodies]
        y = [ag[1] for ag in antibodies]
        scatter(x, y, color = 'yellow')
    axis('scaled')
    axis([0, width, 0, height])
    title('t = ' + str(time))

    subplot(1, 2, 2)
    cla()
    plot(pathogenData, color = 'red')
    plot(markedData, color = 'pink')
    plot(killerData, color = 'green')
    plot(helperData, color = 'blue')
    plot(antibodyData, color = 'yellow')
    title('Populations')

def clip(a, amin, amax):
    if a < amin: return amin
    elif a > amax: return amax
    else: return a

def update():
    global time, pathogens, killers, pathogenData, killerData, helpers, helperData, marked, markedData, antibodies, AntibodyData, Recoveries, Deaths, Result, Results

    time += 1
    
    # simulate random motion
    
    for i in range(len(pathogens)):
        if pathogens[i][2] == 0:
            pathogens[i][0] = pathogens[i][0] + normal(0, pathogenNoiseLevel)
            pathogens[i][1] = pathogens[i][1] + normal(0, pathogenNoiseLevel)
            pathogens[i][0] = clip(pathogens[i][0], 0, width)
            pathogens[i][1] = clip(pathogens[i][1], 0, height)
        else:
            pathogens[i][0] = pathogens[i][0] + 0.3*normal(0, pathogenNoiseLevel)
            pathogens[i][1] = pathogens[i][1] + 0.3*normal(0, pathogenNoiseLevel)
            pathogens[i][0] = clip(pathogens[i][0], 0, width)
            pathogens[i][1] = clip(pathogens[i][1], 0, height)

    #for ag in pathogens:
     #   ag[0] += normal(0, pathogenNoiseLevel)
     #   ag[1] += normal(0, pathogenNoiseLevel)
     #   ag[0] = clip(ag[0], 0, width)
     #   ag[1] = clip(ag[1], 0, height)
     #   print(ag)
    #print('---')
    
    for ag in marked:
        if ag[2] == 0: 
            ag[0] += normal(0, markedNoiseLevel)
            ag[1] += normal(0, markedNoiseLevel)
            ag[0] = clip(ag[0], 0, width)
            ag[1] = clip(ag[1], 0, height)
        else:
            ag[0] += normal(0, 0.3* markedNoiseLevel)
            ag[1] += normal(0, 0.3* markedNoiseLevel)
            ag[0] = clip(ag[0], 0, width)
            ag[1] = clip(ag[1], 0, height) 
        
    for ag in killers:
        ag[0] += normal(0, killerNoiseLevel)
        ag[1] += normal(0, killerNoiseLevel)
        ag[0] = clip(ag[0], 0, width)
        ag[1] = clip(ag[1], 0, height)
        
    for ag in helpers:
        ag[0] += normal(0, helperNoiseLevel)
        ag[1] += normal(0, helperNoiseLevel)
        ag[0] = clip(ag[0], 0, width)
        ag[1] = clip(ag[1], 0, height)
        
    for ag in antibodies:
        ag[0] += normal(0, antibodyNoiseLevel)
        ag[1] += normal(0, antibodyNoiseLevel)
        ag[0] = clip(ag[0], 0, width)
        ag[1] = clip(ag[1], 0, height)    
    
    # detect collision and change state
    killing_locations = []
    for i in range(len(killers)):
        killers[i][2] += 1                      # Killer Cell's age increasing
        for j in range(len(marked)):
            if marked[j] != toBeRemoved:
                if (killers[i][0]-marked[j][0])**2 + (killers[i][1]-marked[j][1])**2 < CDsquared:
                    killing_locations.append([(killers[i][0]+marked[j][0])/2,(killers[i][1]+marked[j][1])/2])
                    marked[j] = toBeRemoved          # Pathogen is Killed
        if killers[i][2] > killerHungerLimit:
            killers[i] = toBeRemoved            # Killer Cell Lifespan Exceeded
    
    marking_locations = []
    for i in range(len(helpers)):
        helpers[i][2] += 1                      # Helper Cell's Age increasing
        for j in range(len(pathogens)):
            if pathogens[j] != toBeRemoved:
                if (helpers[i][0]-pathogens[j][0])**2 + (helpers[i][1]-pathogens[j][1])**2 < CDsquared:
                    marking_locations.append([(helpers[i][0]+pathogens[j][0])/2,(helpers[i][1]+pathogens[j][1])/2])
                    marked.append(pathogens[j]) #Marked Pathogen entity is created
                    pathogens[j] = toBeRemoved # Unmarked Pathogen entity is removed
        if helpers[i][2] > helperHungerLimit:
            helpers[i] = toBeRemoved            # Helper Cell Lifespan Exceeded
            
    for i in range(len(antibodies)):
        antibodies[i][2] += 1                      # Killer Cell's age increasing
        for j in range(len(marked)):
            if marked[j] != toBeRemoved:
                if (antibodies[i][0]-marked[j][0])**2 + (antibodies[i][1]-marked[j][1])**2 < CDsquared:
                    marked[j][2] = 1
            
    for i in range(len(antibodies)):
        for j in range(len(pathogens)):
            if pathogens[j] != toBeRemoved:
                if (antibodies[i][0]-pathogens[j][0])**2 + (antibodies[i][1]-pathogens[j][1])**2 < CDsquared:
                    pathogens[j][2] = 1
        if antibodies[i][2] > antibodyHungerLimit:
            antibodies[i] = toBeRemoved            # Killer Cell Lifespan Exceeded
                    
        
    # remove "toBeRemoved" agents
    while toBeRemoved in pathogens:
        pathogens.remove(toBeRemoved)
    while toBeRemoved in marked:
        marked.remove(toBeRemoved)
    while toBeRemoved in killers:
        killers.remove(toBeRemoved)
    while toBeRemoved in helpers:
        helpers.remove(toBeRemoved)
    while toBeRemoved in antibodies:
        antibodies.remove(toBeRemoved)

    # count survivors' populations
    pathogenPopulation = len(pathogens)
    markedPopulation = len(marked)
    killerPopulation = len(killers)
    helperPopulation = len(helpers)

    # produce offspring
    for i in range(len(pathogens)):
        random2 = random()
        if pathogens[i][2] == 0 and random2 < pathogenReproductionRate or random2/3 <pathogenReproductionRate:
            offspring = pathogens[i][0:2]
            offspring.append(0)
            pathogens.append(offspring) # making and adding a copy of the parent
            pathogenPopulation +=1
    
    for i in range(len(marked)):
        random2 = random()
        if marked[i][2] == 0 and random2 < markedReproductionRate or random2/3 < markedReproductionRate:
            offspring = marked[i][0:2]
            offspring.append(0)
            pathogens.append(offspring) # making and adding a copy of the parent, new cell is unmarked
            pathogenPopulation +=1
    
    ##This loop is important but something about it keeps crashing python
    
    newKillersToday = 0
    while len(killers)<initialKillerPopulation or newKillersToday < min(killerReproductionLimit,len(marking_locations)):
        if len(killing_locations)>1:
            random_choice = killing_locations[int(len(killing_locations)*random()//1)]
            angle = 2*pi*rand()
            #New_X = random_choice[0]+5*rand()*sin(angle)
            #New_Y = random_choice[1]+5*rand()*cos(angle)
            New_X = random_choice[0] + normal(0,5)
            New_Y = random_choice[1] + normal(0,5)
        else:
            New_X = 100*random()
            New_Y = 100*random()
        offspring = [New_X,New_Y,0]
        offspring[0] = clip(offspring[0], 0, width)
        offspring[1] = clip(offspring[1], 0, height)
        killers.append(offspring) # making and adding a copy of the parent
        newKillersToday += 1
        
    
    for i in range(20):
        if len(marking_locations)>1:
            random_choice = marking_locations[int(len(marking_locations)*random()//1)]
            New_X = random_choice[0]+normal(0,5)
            New_Y = random_choice[1]+normal(0,5)
        else:
            New_X = 100*random()
            New_Y = 100*random()
        if len(helpers)<helperPopulationLimit[0] or random() < .5 and len(helpers)<helperPopulationLimit[1] :
            offspring = [New_X,New_Y,0]
            offspring[0] = clip(offspring[0], 0, width)
            offspring[1] = clip(offspring[1], 0, height)
            helpers.append(offspring) # making and adding a copy of the parent
            
    while len(antibodies)<initialAntibodyPopulation:
        New_X = 100*rand()
        New_Y = 100*rand()
        antibodies.append([New_X,New_Y,0])

    pathogenData.append(len(pathogens))
    killerData.append(len(killers))
    helperData.append(len(helpers))
    markedData.append(len(marked))
    antibodyData.append(len(antibodies))
    
    if Result == False:
        if len(marked)+len(pathogens)>250 :
            Result = True
            Deaths += 1
            Results.append(['Death', time])
            print('Died')
        if len(marked)+len(pathogens)==0 :
            Result = True
            Recoveries += 1
            Results.append(['Recovery', time])
            print('Healthy')    


In [21]:
##Unhealthy Individual

import pycxsimulator
from pylab import *

Recoveries = 0
Deaths = 0
Result = False
Results = []
immune_factor = .92

width = 100
height = 100

initialPathogenPopulation = 10
pathogenReproductionRate = 0.02
pathogenPopulationLimit = 10**10
pathogenNoiseLevel = 3

initialMarkedPopulation = 0
markedReproductionRate = .02
markedPopulationLimit = 10**10
markedNoiseLevel = 3

helperPopulationLimit = [int(immune_factor*150),int(immune_factor*175)]
initialHelperPopulation = helperPopulationLimit[0]
helperReproductionLimit = 0.2 ##Not presently used
helperNoiseLevel = 2.5
helperHungerLimit = 10

initialKillerPopulation = int(immune_factor*50)
killerReproductionLimit = int(immune_factor*25)
killerPopulationLimit = int(immune_factor*100)
killerNoiseLevel = 2.5
killerHungerLimit = 10

initialAntibodyPopulation = int(immune_factor*50)
antibodyReproductionLimit = int(immune_factor*25)
antibodyPopulationLimit = int(immune_factor*100)
antibodyNoiseLevel = 2.5
antibodyHungerLimit = 10

collisionDistance = 2.32
CDsquared = collisionDistance ** 2
toBeRemoved = -1

def initialize():
    global time, pathogens, killers, pathogenData, killerData, helpers, helperData, marked, markedData, antibodies ,antibodyData, Result

    time = 0
    Result = False
    
    pathogens = []
    for i in range(initialPathogenPopulation):
        pathogens.append([uniform(0, width), uniform(0, height),0])

    killers = []
    for i in range(initialKillerPopulation):
        killers.append([uniform(0, width), uniform(0, height), 0])
        
    helpers = []
    for i in range(initialHelperPopulation):
        helpers.append([uniform(0, width), uniform(0, height),round(10*random())])   
        
    marked = []
    for i in range(initialMarkedPopulation): #Can Probably be Deleted, Only Useful if starting simulation mid-infection
        Marked.append([uniform(0, width), uniform(0, height),0]) #Can Probably be Deleted, Only Useful if starting simulation mid-infection
        
    antibodies = []
    for i in range(initialAntibodyPopulation):
        antibodies.append([uniform(0, width), uniform(0, height), 0]) 

    pathogenData = [initialPathogenPopulation]
    killerData = [initialKillerPopulation]
    helperData = [initialHelperPopulation]
    markedData = [initialMarkedPopulation]
    antibodyData = [initialAntibodyPopulation]

def observe():
    subplot(1, 2, 1)
    cla()
    if pathogens != []:
        x = [ag[0] for ag in pathogens]
        y = [ag[1] for ag in pathogens]
        scatter(x, y, color = 'red')
    if marked != []:
        x = [ag[0] for ag in marked]
        y = [ag[1] for ag in marked]
        scatter(x, y, color = 'pink')
    if killers != []:
        x = [ag[0] for ag in killers]
        y = [ag[1] for ag in killers]
        scatter(x, y, color = 'green')
    if helpers != []:
        x = [ag[0] for ag in helpers]
        y = [ag[1] for ag in helpers]
        scatter(x, y, color = 'blue')
    if antibodies != []:
        x = [ag[0] for ag in antibodies]
        y = [ag[1] for ag in antibodies]
        scatter(x, y, color = 'yellow')
    axis('scaled')
    axis([0, width, 0, height])
    title('t = ' + str(time))

    subplot(1, 2, 2)
    cla()
    plot(pathogenData, color = 'red')
    plot(markedData, color = 'pink')
    plot(killerData, color = 'green')
    plot(helperData, color = 'blue')
    plot(antibodyData, color = 'yellow')
    title('Populations')

def clip(a, amin, amax):
    if a < amin: return amin
    elif a > amax: return amax
    else: return a

def update():
    global time, pathogens, killers, pathogenData, killerData, helpers, helperData, marked, markedData, antibodies, AntibodyData, Recoveries, Deaths, Result, Results

    time += 1
    
    # simulate random motion
    
    for i in range(len(pathogens)):
        if pathogens[i][2] == 0:
            pathogens[i][0] = pathogens[i][0] + normal(0, pathogenNoiseLevel)
            pathogens[i][1] = pathogens[i][1] + normal(0, pathogenNoiseLevel)
            pathogens[i][0] = clip(pathogens[i][0], 0, width)
            pathogens[i][1] = clip(pathogens[i][1], 0, height)
        else:
            pathogens[i][0] = pathogens[i][0] + 0.3*normal(0, pathogenNoiseLevel)
            pathogens[i][1] = pathogens[i][1] + 0.3*normal(0, pathogenNoiseLevel)
            pathogens[i][0] = clip(pathogens[i][0], 0, width)
            pathogens[i][1] = clip(pathogens[i][1], 0, height)

    #for ag in pathogens:
     #   ag[0] += normal(0, pathogenNoiseLevel)
     #   ag[1] += normal(0, pathogenNoiseLevel)
     #   ag[0] = clip(ag[0], 0, width)
     #   ag[1] = clip(ag[1], 0, height)
     #   print(ag)
    #print('---')
    
    for ag in marked:
        if ag[2] == 0: 
            ag[0] += normal(0, markedNoiseLevel)
            ag[1] += normal(0, markedNoiseLevel)
            ag[0] = clip(ag[0], 0, width)
            ag[1] = clip(ag[1], 0, height)
        else:
            ag[0] += normal(0, 0.3* markedNoiseLevel)
            ag[1] += normal(0, 0.3* markedNoiseLevel)
            ag[0] = clip(ag[0], 0, width)
            ag[1] = clip(ag[1], 0, height) 
        
    for ag in killers:
        ag[0] += normal(0, killerNoiseLevel)
        ag[1] += normal(0, killerNoiseLevel)
        ag[0] = clip(ag[0], 0, width)
        ag[1] = clip(ag[1], 0, height)
        
    for ag in helpers:
        ag[0] += normal(0, helperNoiseLevel)
        ag[1] += normal(0, helperNoiseLevel)
        ag[0] = clip(ag[0], 0, width)
        ag[1] = clip(ag[1], 0, height)
        
    for ag in antibodies:
        ag[0] += normal(0, antibodyNoiseLevel)
        ag[1] += normal(0, antibodyNoiseLevel)
        ag[0] = clip(ag[0], 0, width)
        ag[1] = clip(ag[1], 0, height)    
    
    # detect collision and change state
    killing_locations = []
    for i in range(len(killers)):
        killers[i][2] += 1                      # Killer Cell's age increasing
        for j in range(len(marked)):
            if marked[j] != toBeRemoved:
                if (killers[i][0]-marked[j][0])**2 + (killers[i][1]-marked[j][1])**2 < CDsquared:
                    killing_locations.append([(killers[i][0]+marked[j][0])/2,(killers[i][1]+marked[j][1])/2])
                    marked[j] = toBeRemoved          # Pathogen is Killed
        if killers[i][2] > killerHungerLimit:
            killers[i] = toBeRemoved            # Killer Cell Lifespan Exceeded
    
    marking_locations = []
    for i in range(len(helpers)):
        helpers[i][2] += 1                      # Helper Cell's Age increasing
        for j in range(len(pathogens)):
            if pathogens[j] != toBeRemoved:
                if (helpers[i][0]-pathogens[j][0])**2 + (helpers[i][1]-pathogens[j][1])**2 < CDsquared:
                    marking_locations.append([(helpers[i][0]+pathogens[j][0])/2,(helpers[i][1]+pathogens[j][1])/2])
                    marked.append(pathogens[j]) #Marked Pathogen entity is created
                    pathogens[j] = toBeRemoved # Unmarked Pathogen entity is removed
        if helpers[i][2] > helperHungerLimit:
            helpers[i] = toBeRemoved            # Helper Cell Lifespan Exceeded
            
    for i in range(len(antibodies)):
        antibodies[i][2] += 1                      # Killer Cell's age increasing
        for j in range(len(marked)):
            if marked[j] != toBeRemoved:
                if (antibodies[i][0]-marked[j][0])**2 + (antibodies[i][1]-marked[j][1])**2 < CDsquared:
                    marked[j][2] = 1
            
    for i in range(len(antibodies)):
        for j in range(len(pathogens)):
            if pathogens[j] != toBeRemoved:
                if (antibodies[i][0]-pathogens[j][0])**2 + (antibodies[i][1]-pathogens[j][1])**2 < CDsquared:
                    pathogens[j][2] = 1
        if antibodies[i][2] > antibodyHungerLimit:
            antibodies[i] = toBeRemoved            # Killer Cell Lifespan Exceeded
                    
        
    # remove "toBeRemoved" agents
    while toBeRemoved in pathogens:
        pathogens.remove(toBeRemoved)
    while toBeRemoved in marked:
        marked.remove(toBeRemoved)
    while toBeRemoved in killers:
        killers.remove(toBeRemoved)
    while toBeRemoved in helpers:
        helpers.remove(toBeRemoved)
    while toBeRemoved in antibodies:
        antibodies.remove(toBeRemoved)

    # count survivors' populations
    pathogenPopulation = len(pathogens)
    markedPopulation = len(marked)
    killerPopulation = len(killers)
    helperPopulation = len(helpers)

    # produce offspring
    for i in range(len(pathogens)):
        random2 = random()
        if pathogens[i][2] == 0 and random2 < pathogenReproductionRate or random2/3 <pathogenReproductionRate:
            offspring = pathogens[i][0:2]
            offspring.append(0)
            pathogens.append(offspring) # making and adding a copy of the parent
            pathogenPopulation +=1
    
    for i in range(len(marked)):
        random2 = random()
        if marked[i][2] == 0 and random2 < markedReproductionRate or random2/3 < markedReproductionRate:
            offspring = marked[i][0:2]
            offspring.append(0)
            pathogens.append(offspring) # making and adding a copy of the parent, new cell is unmarked
            pathogenPopulation +=1
    
    ##This loop is important but something about it keeps crashing python
    
    newKillersToday = 0
    while len(killers)<initialKillerPopulation or newKillersToday < min(killerReproductionLimit,len(marking_locations)):
        if len(killing_locations)>1:
            random_choice = killing_locations[int(len(killing_locations)*random()//1)]
            angle = 2*pi*rand()
            #New_X = random_choice[0]+5*rand()*sin(angle)
            #New_Y = random_choice[1]+5*rand()*cos(angle)
            New_X = random_choice[0] + normal(0,5)
            New_Y = random_choice[1] + normal(0,5)
        else:
            New_X = 100*random()
            New_Y = 100*random()
        offspring = [New_X,New_Y,0]
        offspring[0] = clip(offspring[0], 0, width)
        offspring[1] = clip(offspring[1], 0, height)
        killers.append(offspring) # making and adding a copy of the parent
        newKillersToday += 1
        
    
    for i in range(20):
        if len(marking_locations)>1:
            random_choice = marking_locations[int(len(marking_locations)*random()//1)]
            New_X = random_choice[0]+normal(0,5)
            New_Y = random_choice[1]+normal(0,5)
        else:
            New_X = 100*random()
            New_Y = 100*random()
        if len(helpers)<helperPopulationLimit[0] or random() < .5 and len(helpers)<helperPopulationLimit[1] :
            offspring = [New_X,New_Y,0]
            offspring[0] = clip(offspring[0], 0, width)
            offspring[1] = clip(offspring[1], 0, height)
            helpers.append(offspring) # making and adding a copy of the parent
            
    while len(antibodies)<initialAntibodyPopulation:
        New_X = 100*rand()
        New_Y = 100*rand()
        antibodies.append([New_X,New_Y,0])

    pathogenData.append(len(pathogens))
    killerData.append(len(killers))
    helperData.append(len(helpers))
    markedData.append(len(marked))
    antibodyData.append(len(antibodies))
    
    if Result == False:
        if len(marked)+len(pathogens)>250 :
            Result = True
            Deaths += 1
            Results.append(['Death', time])
            print('Died')
        if len(marked)+len(pathogens)==0 :
            Result = True
            Recoveries += 1
            Results.append(['Recovery', time])
            print('Healthy')    


In [23]:
##Ultrahealthy Individual

import pycxsimulator
from pylab import *

Recoveries = 0
Deaths = 0
Result = False
Results = []

width = 100
height = 100

initialPathogenPopulation = 10
pathogenReproductionRate = 0.02
pathogenPopulationLimit = 10**10
pathogenNoiseLevel = 3

initialMarkedPopulation = 0
markedReproductionRate = .02
markedPopulationLimit = 10**10
markedNoiseLevel = 3

helperPopulationLimit = [150,175]
initialHelperPopulation = helperPopulationLimit[0]
helperReproductionLimit = 0.2 ##Not presently used
helperNoiseLevel = 2.5
helperHungerLimit = 10

initialKillerPopulation = 50
killerReproductionLimit = 25
killerPopulationLimit = 100
killerNoiseLevel = 2.5
killerHungerLimit = 10

initialAntibodyPopulation = int(1.5*50)
antibodyReproductionLimit = int(1.5*25)
antibodyPopulationLimit = int(1.5*100)
antibodyNoiseLevel = 2.5
antibodyHungerLimit = 10

collisionDistance = 2.32
CDsquared = collisionDistance ** 2
toBeRemoved = -1

def initialize():
    global time, pathogens, killers, pathogenData, killerData, helpers, helperData, marked, markedData, antibodies ,antibodyData, Result

    time = 0
    Result = False
    
    pathogens = []
    for i in range(initialPathogenPopulation):
        pathogens.append([uniform(0, width), uniform(0, height),0])

    killers = []
    for i in range(initialKillerPopulation):
        killers.append([uniform(0, width), uniform(0, height), 0])
        
    helpers = []
    for i in range(initialHelperPopulation):
        helpers.append([uniform(0, width), uniform(0, height),round(10*random())])   
        
    marked = []
    for i in range(initialMarkedPopulation): #Can Probably be Deleted, Only Useful if starting simulation mid-infection
        Marked.append([uniform(0, width), uniform(0, height),0]) #Can Probably be Deleted, Only Useful if starting simulation mid-infection
        
    antibodies = []
    for i in range(initialAntibodyPopulation):
        antibodies.append([uniform(0, width), uniform(0, height), 0]) 

    pathogenData = [initialPathogenPopulation]
    killerData = [initialKillerPopulation]
    helperData = [initialHelperPopulation]
    markedData = [initialMarkedPopulation]
    antibodyData = [initialAntibodyPopulation]

def observe():
    subplot(1, 2, 1)
    cla()
    if pathogens != []:
        x = [ag[0] for ag in pathogens]
        y = [ag[1] for ag in pathogens]
        scatter(x, y, color = 'red')
    if marked != []:
        x = [ag[0] for ag in marked]
        y = [ag[1] for ag in marked]
        scatter(x, y, color = 'pink')
    if killers != []:
        x = [ag[0] for ag in killers]
        y = [ag[1] for ag in killers]
        scatter(x, y, color = 'green')
    if helpers != []:
        x = [ag[0] for ag in helpers]
        y = [ag[1] for ag in helpers]
        scatter(x, y, color = 'blue')
    if antibodies != []:
        x = [ag[0] for ag in antibodies]
        y = [ag[1] for ag in antibodies]
        scatter(x, y, color = 'yellow')
    axis('scaled')
    axis([0, width, 0, height])
    title('t = ' + str(time))

    subplot(1, 2, 2)
    cla()
    plot(pathogenData, color = 'red')
    plot(markedData, color = 'pink')
    plot(killerData, color = 'green')
    plot(helperData, color = 'blue')
    plot(antibodyData, color = 'yellow')
    title('Populations')

def clip(a, amin, amax):
    if a < amin: return amin
    elif a > amax: return amax
    else: return a

def update():
    global time, pathogens, killers, pathogenData, killerData, helpers, helperData, marked, markedData, antibodies, AntibodyData, Recoveries, Deaths, Result, Results

    time += 1
    
    # simulate random motion
    
    for i in range(len(pathogens)):
        if pathogens[i][2] == 0:
            pathogens[i][0] = pathogens[i][0] + normal(0, pathogenNoiseLevel)
            pathogens[i][1] = pathogens[i][1] + normal(0, pathogenNoiseLevel)
            pathogens[i][0] = clip(pathogens[i][0], 0, width)
            pathogens[i][1] = clip(pathogens[i][1], 0, height)
        else:
            pathogens[i][0] = pathogens[i][0] + 0.3*normal(0, pathogenNoiseLevel)
            pathogens[i][1] = pathogens[i][1] + 0.3*normal(0, pathogenNoiseLevel)
            pathogens[i][0] = clip(pathogens[i][0], 0, width)
            pathogens[i][1] = clip(pathogens[i][1], 0, height)

    #for ag in pathogens:
     #   ag[0] += normal(0, pathogenNoiseLevel)
     #   ag[1] += normal(0, pathogenNoiseLevel)
     #   ag[0] = clip(ag[0], 0, width)
     #   ag[1] = clip(ag[1], 0, height)
     #   print(ag)
    #print('---')
    
    for ag in marked:
        if ag[2] == 0: 
            ag[0] += normal(0, markedNoiseLevel)
            ag[1] += normal(0, markedNoiseLevel)
            ag[0] = clip(ag[0], 0, width)
            ag[1] = clip(ag[1], 0, height)
        else:
            ag[0] += normal(0, 0.3* markedNoiseLevel)
            ag[1] += normal(0, 0.3* markedNoiseLevel)
            ag[0] = clip(ag[0], 0, width)
            ag[1] = clip(ag[1], 0, height) 
        
    for ag in killers:
        ag[0] += normal(0, killerNoiseLevel)
        ag[1] += normal(0, killerNoiseLevel)
        ag[0] = clip(ag[0], 0, width)
        ag[1] = clip(ag[1], 0, height)
        
    for ag in helpers:
        ag[0] += normal(0, helperNoiseLevel)
        ag[1] += normal(0, helperNoiseLevel)
        ag[0] = clip(ag[0], 0, width)
        ag[1] = clip(ag[1], 0, height)
        
    for ag in antibodies:
        ag[0] += normal(0, antibodyNoiseLevel)
        ag[1] += normal(0, antibodyNoiseLevel)
        ag[0] = clip(ag[0], 0, width)
        ag[1] = clip(ag[1], 0, height)    
    
    # detect collision and change state
    killing_locations = []
    for i in range(len(killers)):
        killers[i][2] += 1                      # Killer Cell's age increasing
        for j in range(len(marked)):
            if marked[j] != toBeRemoved:
                if (killers[i][0]-marked[j][0])**2 + (killers[i][1]-marked[j][1])**2 < CDsquared:
                    killing_locations.append([(killers[i][0]+marked[j][0])/2,(killers[i][1]+marked[j][1])/2])
                    marked[j] = toBeRemoved          # Pathogen is Killed
        if killers[i][2] > killerHungerLimit:
            killers[i] = toBeRemoved            # Killer Cell Lifespan Exceeded
    
    marking_locations = []
    for i in range(len(helpers)):
        helpers[i][2] += 1                      # Helper Cell's Age increasing
        for j in range(len(pathogens)):
            if pathogens[j] != toBeRemoved:
                if (helpers[i][0]-pathogens[j][0])**2 + (helpers[i][1]-pathogens[j][1])**2 < CDsquared:
                    marking_locations.append([(helpers[i][0]+pathogens[j][0])/2,(helpers[i][1]+pathogens[j][1])/2])
                    marked.append(pathogens[j]) #Marked Pathogen entity is created
                    pathogens[j] = toBeRemoved # Unmarked Pathogen entity is removed
        if helpers[i][2] > helperHungerLimit:
            helpers[i] = toBeRemoved            # Helper Cell Lifespan Exceeded
            
    for i in range(len(antibodies)):
        antibodies[i][2] += 1                      # Killer Cell's age increasing
        for j in range(len(marked)):
            if marked[j] != toBeRemoved:
                if (antibodies[i][0]-marked[j][0])**2 + (antibodies[i][1]-marked[j][1])**2 < CDsquared:
                    marked[j][2] = 1
            
    for i in range(len(antibodies)):
        for j in range(len(pathogens)):
            if pathogens[j] != toBeRemoved:
                if (antibodies[i][0]-pathogens[j][0])**2 + (antibodies[i][1]-pathogens[j][1])**2 < CDsquared:
                    pathogens[j][2] = 1
        if antibodies[i][2] > antibodyHungerLimit:
            antibodies[i] = toBeRemoved            # Killer Cell Lifespan Exceeded
                    
        
    # remove "toBeRemoved" agents
    while toBeRemoved in pathogens:
        pathogens.remove(toBeRemoved)
    while toBeRemoved in marked:
        marked.remove(toBeRemoved)
    while toBeRemoved in killers:
        killers.remove(toBeRemoved)
    while toBeRemoved in helpers:
        helpers.remove(toBeRemoved)
    while toBeRemoved in antibodies:
        antibodies.remove(toBeRemoved)

    # count survivors' populations
    pathogenPopulation = len(pathogens)
    markedPopulation = len(marked)
    killerPopulation = len(killers)
    helperPopulation = len(helpers)

    # produce offspring
    for i in range(len(pathogens)):
        random2 = random()
        if pathogens[i][2] == 0 and random2 < pathogenReproductionRate or random2/3 <pathogenReproductionRate:
            offspring = pathogens[i][0:2]
            offspring.append(0)
            pathogens.append(offspring) # making and adding a copy of the parent
            pathogenPopulation +=1
    
    for i in range(len(marked)):
        random2 = random()
        if marked[i][2] == 0 and random2 < markedReproductionRate or random2/3 < markedReproductionRate:
            offspring = marked[i][0:2]
            offspring.append(0)
            pathogens.append(offspring) # making and adding a copy of the parent, new cell is unmarked
            pathogenPopulation +=1
    
    ##This loop is important but something about it keeps crashing python
    
    newKillersToday = 0
    while len(killers)<initialKillerPopulation or newKillersToday < min(killerReproductionLimit,len(marking_locations)):
        if len(killing_locations)>1:
            random_choice = killing_locations[int(len(killing_locations)*random()//1)]
            angle = 2*pi*rand()
            #New_X = random_choice[0]+5*rand()*sin(angle)
            #New_Y = random_choice[1]+5*rand()*cos(angle)
            New_X = random_choice[0] + normal(0,5)
            New_Y = random_choice[1] + normal(0,5)
        else:
            New_X = 100*random()
            New_Y = 100*random()
        offspring = [New_X,New_Y,0]
        offspring[0] = clip(offspring[0], 0, width)
        offspring[1] = clip(offspring[1], 0, height)
        killers.append(offspring) # making and adding a copy of the parent
        newKillersToday += 1
        
    
    for i in range(20):
        if len(marking_locations)>1:
            random_choice = marking_locations[int(len(marking_locations)*random()//1)]
            New_X = random_choice[0]+normal(0,5)
            New_Y = random_choice[1]+normal(0,5)
        else:
            New_X = 100*random()
            New_Y = 100*random()
        if len(helpers)<helperPopulationLimit[0] or random() < .5 and len(helpers)<helperPopulationLimit[1] :
            offspring = [New_X,New_Y,0]
            offspring[0] = clip(offspring[0], 0, width)
            offspring[1] = clip(offspring[1], 0, height)
            helpers.append(offspring) # making and adding a copy of the parent
            
    while len(antibodies)<initialAntibodyPopulation:
        New_X = 100*rand()
        New_Y = 100*rand()
        antibodies.append([New_X,New_Y,0])

    pathogenData.append(len(pathogens))
    killerData.append(len(killers))
    helperData.append(len(helpers))
    markedData.append(len(marked))
    antibodyData.append(len(antibodies))
    
    if Result == False:
        if len(marked)+len(pathogens)>250 :
            Result = True
            Deaths += 1
            Results.append(['Death', time])
            print('Died')
        if len(marked)+len(pathogens)==0 :
            Result = True
            Recoveries += 1
            Results.append(['Recovery', time])
            print('Healthy',len(Results))    


In [25]:
pycxsimulator.GUI().start(func=[initialize, observe, update])

Died
Died
Healthy 4


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\asdf2\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\asdf2\anaconda3\lib\site-packages\matplotlib\backends\_backend_tk.py", line 478, in destroy
    Gcf.destroy(self)
  File "C:\Users\asdf2\anaconda3\lib\site-packages\matplotlib\_pylab_helpers.py", line 67, in destroy
    manager.destroy()
  File "C:\Users\asdf2\anaconda3\lib\site-packages\matplotlib\backends\_backend_tk.py", line 495, in destroy
    self._window_dpi.trace_remove('write', self._window_dpi_cbname)
  File "C:\Users\asdf2\anaconda3\lib\tkinter\__init__.py", line 430, in trace_remove
    self._tk.deletecommand(cbname)
_tkinter.TclError: can't delete Tcl command


In [24]:
Recoveries = 0
Deaths = 0
Results = []
Result = False
while len(Results)<1000:
    initialize()
    while Result == False:
        update()
        
print("Recoveries:",Recoveries)
print("Deaths:",Deaths)

Healthy 1


KeyboardInterrupt: 

In [17]:
timeh, timeu = 0,0
Healthy_Times = []
Death_Times = []

for a in Results:
    print(a)
    if a[0] == 'Recovery':
        timeh += a[1]
        Healthy_Times.append(a[1])
    else:
        timeu += a[1]
        Death_Times.append(a[1])

print(timeh/Recoveries)
print(timeu/Deaths)

['Recovery', 289]
['Recovery', 206]
['Death', 682]
['Recovery', 713]
['Death', 601]
['Death', 678]
['Recovery', 237]
['Death', 253]
['Recovery', 1294]
['Recovery', 139]
['Death', 952]
['Death', 294]
['Death', 232]
['Recovery', 128]
['Recovery', 196]
['Recovery', 275]
['Recovery', 242]
['Recovery', 198]
['Death', 413]
['Recovery', 572]
['Recovery', 552]
['Recovery', 298]
['Recovery', 609]
['Recovery', 293]
['Recovery', 1319]
['Death', 683]
['Recovery', 198]
['Death', 1443]
['Recovery', 465]
['Recovery', 317]
['Recovery', 322]
['Recovery', 405]
['Recovery', 391]
['Recovery', 164]
['Recovery', 391]
['Recovery', 155]
['Recovery', 1177]
['Recovery', 665]
['Recovery', 210]
['Recovery', 551]
['Recovery', 228]
['Death', 538]
['Death', 596]
['Recovery', 508]
['Recovery', 1729]
['Recovery', 280]
['Death', 797]
['Death', 912]
['Recovery', 442]
['Death', 539]
['Death', 575]
['Recovery', 140]
['Recovery', 739]
['Recovery', 1012]
['Recovery', 53]
['Recovery', 709]
['Death', 411]
['Death', 495]
['Dea

In [18]:
print(min(Healthy_Times),max(Healthy_Times))
print(min(Death_Times),max(Death_Times))

27 2731
146 2812
